In [1]:
import pandas as pd
import os
import json
import numpy as np
import re
from operator import itemgetter
from glob import glob
import itertools
import csv


## Tokens

In [4]:
strains = "/home/gomez/gomez/preds2/NER_output/STRAIN/805.parquet"

In [5]:
df = pd.read_parquet(strains).explode("ner").dropna()

FileNotFoundError: [Errno 2] No such file or directory: '/home/gomez/gomez/preds2/NER_output/STRAIN/805.parquet'

In [201]:
grouped = df.groupby("text").agg({"ner": lambda x: list(x)}).reset_index()

In [203]:
def merge_entities(entity_list):
    merged_list = []
    skip = False

    for i in range(len(entity_list)):
        if skip:
            skip = False
            continue

        current_entity = entity_list[i]
        current_entity.pop('word', None)  # Remove 'word' entry
        if current_entity['entity_group'] == 'B':
            scores = [current_entity['score']]
            # Look ahead to find contiguous 'I' entities
            next_idx = i + 1
            while next_idx < len(entity_list) and \
                  entity_list[next_idx]['entity_group'] == 'I' and \
                  (entity_list[next_idx]['start'] - current_entity['end']) <= 4:
                scores.append(entity_list[next_idx]['score'])
                current_entity['end'] = entity_list[next_idx]['end']  # Update end value
                skip = True
                next_idx += 1

            # Average the scores
            current_entity['score'] = sum(scores) / len(scores)

        merged_list.append(current_entity)

    return merged_list


In [204]:
grouped['ner'] = grouped['ner'].apply(merge_entities)


In [206]:
grouped = grouped.explode("ner")

In [207]:
grouped

,text,ner
0,"""Coordinate positions or positions of three re...","{'end': 159, 'entity_group': 'B', 'score': 0.6..."
1,"#MW113) were used, and stored in 0.5-0.75 mL o...","{'end': 6, 'entity_group': 'B', 'score': 0.880..."
2,"& Hur (paratype: KBA-L-0001037); South Korea, ...","{'end': 27, 'entity_group': 'B', 'score': 0.78..."
3,"(1) With ectopic NSP1 expression, we observed ...","{'end': 161, 'entity_group': 'B', 'score': 0.7..."
4,(1)Culture Pmet3-CDC20 Pgal-CEN3-tetOs TetR-GF...,"{'end': 66, 'entity_group': 'B', 'score': 0.75..."
...,...,...
9410,经Xho1、EcoR1酶切图筛选Track-Eazyp53(del)、Track-Easy-...,"{'end': 9, 'entity_group': 'B', 'score': 0.658..."
9411,继发T790M突变建议的主要检测手段有cobas、AMRS、ddPCR甚至二代测序（next...,"{'end': 4, 'entity_group': 'B', 'score': 0.632..."
9412,考虑到MPOG-463A在不同种族的差异分布(http://www.ncbi.nlm.nih...,"{'end': 12, 'entity_group': 'B', 'score': 0.78..."
9413,肺癌细胞系A549（CCL#185）购自American Type Culture Coll...,"{'end': 9, 'entity_group': 'B', 'score': 0.773..."


In [208]:
grouped = pd.concat(
                [grouped.drop(columns="ner"), grouped.ner.apply(pd.Series)], axis=1
            )

In [209]:
grouped

,text,end,entity_group,score,start
0,"""Coordinate positions or positions of three re...",159,B,0.652696,153
1,"#MW113) were used, and stored in 0.5-0.75 mL o...",6,B,0.880458,1
2,"& Hur (paratype: KBA-L-0001037); South Korea, ...",27,B,0.789787,17
3,"(1) With ectopic NSP1 expression, we observed ...",161,B,0.705191,158
4,(1)Culture Pmet3-CDC20 Pgal-CEN3-tetOs TetR-GF...,66,B,0.757170,64
...,...,...,...,...,...
9410,经Xho1、EcoR1酶切图筛选Track-Eazyp53(del)、Track-Easy-...,9,B,0.658696,6
9411,继发T790M突变建议的主要检测手段有cobas、AMRS、ddPCR甚至二代测序（next...,4,B,0.632785,2
9412,考虑到MPOG-463A在不同种族的差异分布(http://www.ncbi.nlm.nih...,12,B,0.780138,3
9413,肺癌细胞系A549（CCL#185）购自American Type Culture Coll...,9,B,0.773091,5


In [210]:
grouped['word'] = grouped.apply(lambda row: row['text'][row['start']:row['end']], axis=1)


In [212]:
grouped["word"] = grouped["word"].str.lower()

In [214]:
grouped.to_csv("test.csv")

---

In [4]:
data = "810"
strains = glob(f"/home/gomez/gomez/preds{data}/NER_output/STRAIN/*.parquet")

In [7]:
def merge_entities(entity_list):
    merged_list = []
    skip = False

    for i in range(len(entity_list)):
        if skip:
            skip = False
            continue

        current_entity = entity_list[i]
        current_entity.pop("word", None)  # Remove 'word' entry
        if current_entity["entity_group"] == "B":
            scores = [current_entity["score"]]
            # Look ahead to find contiguous 'I' entities
            next_idx = i + 1
            while (
                next_idx < len(entity_list)
                and entity_list[next_idx]["entity_group"] == "I"
                and (entity_list[next_idx]["start"] - current_entity["end"]) <= 4
            ):
                scores.append(entity_list[next_idx]["score"])
                current_entity["end"] = entity_list[next_idx]["end"]  # Update end value
                skip = True
                next_idx += 1

            # Average the scores
            current_entity["score"] = sum(scores) / len(scores)

        merged_list.append(current_entity)

    return merged_list

In [8]:
l = []
for file in tqdm(strains):
	d = pd.read_parquet(file).explode("ner").dropna()
	grouped = d.groupby("text").agg({"ner": lambda x: list(x)}).reset_index()
	grouped["ner"] = grouped["ner"].apply(merge_entities)
	grouped = grouped.explode("ner")
	grouped = pd.concat(
		[grouped.drop(columns="ner"), grouped.ner.apply(pd.Series)], axis=1
	)
	l.append(grouped)
df = pd.concat(l)

In [10]:
df["word"] = df.apply(
	lambda row: row["text"][row["start"] : row["end"]], axis=2
)

In [15]:
df_sorted = df.sort_values("word")

In [20]:
df["word"] = df["word"].str.lower()

In [23]:
df.sort_values("text").iloc[0,0]

'"64-111 K 2D pass reads (compared with 21 K for the typical R7.3 experiment)".'

## Check most common entities per category

In [2]:
data = "810"
def merge_entities(entity_list):
    merged_list = []
    skip = False

    for i in range(len(entity_list)):
        if skip:
            skip = False
            continue

        current_entity = entity_list[i]
        current_entity.pop("word", None)  # Remove 'word' entry
        if current_entity["entity_group"] == "B":
            scores = [current_entity["score"]]
            # Look ahead to find contiguous 'I' entities
            next_idx = i + 1
            while (
                next_idx < len(entity_list)
                and entity_list[next_idx]["entity_group"] == "I"
                and (entity_list[next_idx]["start"] - current_entity["end"]) <= 4
            ):
                scores.append(entity_list[next_idx]["score"])
                current_entity["end"] = entity_list[next_idx]["end"]  # Update end value
                skip = True
                next_idx += 1

            # Average the scores
            current_entity["score"] = sum(scores) / len(scores)

        merged_list.append(current_entity)

    return merged_list
path = f"/home/gomez/gomez/preds{data}/NER_output"

In [7]:
d = pd.read_parquet(f"{path}/COMPOUND.parquet").explode("ner").dropna()

In [8]:
grouped = d.groupby("text").agg({"ner": lambda x: list(x)}).reset_index()
grouped["ner"] = grouped["ner"].apply(merge_entities)
grouped = grouped.explode("ner")
grouped = pd.concat(
                [grouped.drop(columns="ner"), grouped.ner.apply(pd.Series)], axis=1
            )

In [10]:
grouped["word"] = grouped.apply(
            lambda row: row["text"][row["start"] : row["end"]], axis=1
        )

In [31]:
grouped.value_counts("word").head(300).reset_index().word.values

array(['2', 'CO_', 'PBS', 'CO_2', 'NaCl', 'H_2O_2', 'DMSO', 'cisplatin',
       'DOX', 'doxorubicin', 'EDTA', 'HEPES', 'iron', 'Cd', 'PMA', '5-FU',
       'phosphate', 'P', 'N', 'H_', 'TMZ', 'Fe', 'oxygen', 'Dox', '4',
       'ciprofloxacin', 'HCl', 'ethanol', 'paclitaxel', 'PI', 'water',
       'paraformaldehyde', 'SDS', 'ROS', 'gentamicin', 'Ca^2+',
       'tetracycline', 'Tris', 'methanol', 'nitrogen', 'copper', 'KCl',
       'Zn', 'CaCl_2', 'curcumin', 'PTX', 'MgCl_2', 'Triton X-100',
       'FITC', 'Cu', 'colistin', '3', 'O_2', 'chloroquine', '5', 'BSA',
       'doxycycline', 'PEG', '2O', 'LPS', 'hydrogen peroxide', 'NAC',
       'CDDP', 'CQ', 'gemcitabine', 'DTT', 'formaldehyde', 'K', 'zinc',
       'EGCG', 'MeOH', 'nitrate', 'O_', 'CO', 'gefitinib', 'MgSO_4',
       'salt', 'ampicillin', 'NO', '-', 'AgNPs', 'rifampicin', 'Na', 'Al',
       'calcium', 'Mn', 'Lipofectamine', 'quercetin', 'Ca', 'Tris-',
       'phenol', 'tamoxifen', '% CO_2', 'PE', 'propidium iodide', 'metal',
    

In [35]:
word_list = ['2', 'CO_', 'H_', "4", "UV","-", "% CO_2","3", "6", "1","7","D"]
sample_row = grouped[grouped['word'].isin(word_list)].groupby("word").apply(lambda df: df.sample(1))




In [42]:
with open("sentences.txt", "w") as file:
	for sentence in sample_row["text"].values:
		file.write(sentence + "\n")

---

In [43]:
d = pd.read_parquet(f"{path}/ISOLATE.parquet").explode("ner").dropna()
grouped = d.groupby("text").agg({"ner": lambda x: list(x)}).reset_index()
grouped["ner"] = grouped["ner"].apply(merge_entities)
grouped = grouped.explode("ner")
grouped = pd.concat(
                [grouped.drop(columns="ner"), grouped.ner.apply(pd.Series)], axis=1
            )
grouped["word"] = grouped.apply(
            lambda row: row["text"][row["start"] : row["end"]], axis=1
        )

In [45]:
grouped.value_counts("word").head(200).reset_index().word.values

array(['soil', 'soil sample', 'blood', 'soil samples', 'feces',
       'rhizosphere', 'patient', 'mangrove', 'rhizosphere soil',
       'seawater', 'water', 'food', 'sediment', 'marine sediment',
       'wastewater', 'kimchi', 'fecal', 'hot spring', 'wine', 'sourdough',
       'milk', 'cheese', 'urine', 'human feces', 'sewage', 'soils',
       'dairy', 'sediments', 'roots', 'leaves', 'fecal samples',
       'freshwater', 'deep-sea sediment', 'root', 'sputum', 'stool',
       'gut', 'human', 'marine sediments', 'skin', 'marine',
       'lymph node metastasis', 'activated sludge', 'rumen', 'pig',
       'nasal', 'chicken', 'rhizospheric soil', 'forest soil', 'poultry',
       'environmental', 'compost', 'sediment sample', 'bone marrow',
       'leaf', 'lymph node', 'blood culture', 'meat', 'deep-sea',
       'patients', 'raw milk', 'spleen', 'marine sponge', 'rice',
       'bovine mastitis', '-', 'water sample', 'water samples',
       'human patient', 'chicken meat', 'pigs', 'fermented 

In [46]:
word_list = ["-", "lymph node metastasis","bovine mastitis","leaves of"]
sample_row = grouped[grouped['word'].isin(word_list)].groupby("word").apply(lambda df: df.sample(1))


In [47]:
with open("sentences.txt", "w") as file:
	for sentence in sample_row["text"].values:
		file.write(sentence + "\n")

---

In [48]:
d = pd.read_parquet(f"{path}/SPECIES.parquet").explode("ner").dropna()
grouped = d.groupby("text").agg({"ner": lambda x: list(x)}).reset_index()
grouped["ner"] = grouped["ner"].apply(merge_entities)
grouped = grouped.explode("ner")
grouped = pd.concat(
                [grouped.drop(columns="ner"), grouped.ner.apply(pd.Series)], axis=1
            )
grouped["word"] = grouped.apply(
            lambda row: row["text"][row["start"] : row["end"]], axis=1
        )

In [52]:
grouped.value_counts("word").head(180).reset_index().word.values

array(['E. coli', 'S. aureus', 'Escherichia coli', 'P. aeruginosa', 'S',
       'S.', 'C. albicans', 'P', 'C', 'S. cerevisiae', 'K. pneumoniae',
       'E', 'B', 'Staphylococcus aureus', 'Typhimurium', 'B. subtilis',
       'C. elegans', 'L. monocytogenes', 'A. baumannii', 'A', 'H. pylori',
       'M. tuberculosis', 'V. cholerae', 'C. difficile', 'C. jejuni',
       'P. falciparum', 'Pseudomonas aeruginosa', 'L', 'E. faecalis',
       'L. plantarum', 'M', 'M. bovis', 'S. pneumoniae',
       'Bacillus subtilis', 'coli', 'B. cereus', 'T', 'A. thaliana',
       'T. cruzi', 'Enteritidis', 'B. cinerea', 'S. epidermidis', '.',
       'A. fumigatus', 'N', 'S. mutans', 'Candida albicans', 'T. gondii',
       'P. gingivalis', 'S. enterica', 'Ae. aegypti', 'L. reuteri',
       'S. flexneri', 'V', 'Saccharomyces cerevisiae',
       'Salmonella enterica serovar Typhimurium', 'M. smegmatis', 'H',
       'L. pneumophila', 'Agrobacterium tumefaciens', 'R',
       'Klebsiella pneumoniae', 'L. lactis',

In [53]:
word_list = ["S","S.","P","C","M","E.","Y","coli","E","B","T","V","N","H","R","Typhi","G","B.","C.","sp","K","O"]
sample_row = grouped[grouped['word'].isin(word_list)].groupby("word").apply(lambda df: df.sample(1))

In [54]:
with open("sentences.txt", "w") as file:
	for sentence in sample_row["text"].values:
		file.write(sentence + "\n")

---

In [55]:
d = pd.read_parquet(f"{path}/METABOLITE.parquet").explode("ner").dropna()
grouped = d.groupby("text").agg({"ner": lambda x: list(x)}).reset_index()
grouped["ner"] = grouped["ner"].apply(merge_entities)
grouped = grouped.explode("ner")
grouped = pd.concat(
                [grouped.drop(columns="ner"), grouped.ner.apply(pd.Series)], axis=1
            )
grouped["word"] = grouped.apply(
            lambda row: row["text"][row["start"] : row["end"]], axis=1
        )

In [61]:
grouped.value_counts("word").head(280).reset_index().word.values

array(['streptomycin', 'penicillin', 'glucose', 'LPS', 'L-glutamine',
       'ampicillin', 'ethanol', 'glycerol', 'glutamine', 'sucrose',
       'kanamycin', 'EPS', 'xylose', 'amino', '-', 'acetate',
       'chloramphenicol', 'ATP', 'pyruvate', 'IAA', 'gentamicin',
       'lactate', 'ABA', 'tetracycline', 'starch', 'l-glutamine', '2',
       'SA', 'curcumin', 'insulin', 'galactose', 'proline', 'fructose',
       'glutamate', 'vancomycin', 'Streptomycin', 'EGF', 'Penicillin',
       'PMA', 'erythromycin', 'puromycin', 'arabinose', 'lactose',
       'sodium pyruvate', 'cellulose', 'colistin', 'butyrate',
       'ciprofloxacin', '5-FU', 'cholesterol', 'DHA', 'tryptophan',
       'resveratrol', '5', 'paclitaxel', 'TNF', 'acetic acid', 'arginine',
       'quercetin', 'lipopolysaccharide', 'GA', 'lignin', 'doxorubicin',
       'lactic acid', 'lysine', 'metformin', 'JA', 'GSH', 'PA', 'PHB',
       'rifampicin', 'GABA', 'sugar', 'glycine', 'L', 'doxycycline',
       'yeast', 'FA', 'DON', 'gemc

In [62]:
word_list = ["-","2","5","yeast","L","3","β","6","fatty","1","acid","α","L-","C","B","N","D-","3-","poly","S"]
sample_row = grouped[grouped['word'].isin(word_list)].groupby("word").apply(lambda df: df.sample(1))

In [63]:
with open("sentences.txt", "w") as file:
	for sentence in sample_row["text"].values:
		file.write(sentence + "\n")

---

In [64]:
d = pd.read_parquet(f"{path}/ORGANISM.parquet").explode("ner").dropna()
grouped = d.groupby("text").agg({"ner": lambda x: list(x)}).reset_index()
grouped["ner"] = grouped["ner"].apply(merge_entities)
grouped = grouped.explode("ner")
grouped = pd.concat(
                [grouped.drop(columns="ner"), grouped.ner.apply(pd.Series)], axis=1
            )
grouped["word"] = grouped.apply(
            lambda row: row["text"][row["start"] : row["end"]], axis=1
        )

In [69]:
grouped.value_counts("word").head(330).reset_index().word.values

array(['mice', 'MCF', 'SH', 'MB', 'C57BL', 'SH-SY5Y', 'HEK', 'C57BL/6',
       'Col', 'MDA', 'PC', 'C. elegans', 'N', 'CHO', 'rice',
       'C57BL/6 mice', 'mouse', 'SY5Y', 'S', 'wheat', 'Arabidopsis',
       'BALB/c mice', 'A', '6', 'tomato', '7', 'Vero', 'BALB/c', '2',
       'BALB', 'maize', 'Mice', 'B6', 'T', 'pigs', 'nude', 'chickens',
       'C', 'Caco', 'soybean', 'HT', 'A549', 'C57BL/6J', 'Col-0',
       'RAW264', 'SK', '4T1', 'chicken', 'P', 'cattle', 'B16', 'C6',
       'nude mice', 'BT', 'NOD', 'HK', 'week', 'HepG2', 'C4', 'C57BL/',
       'DF', 'wild', 'NIH', '3T3', 'D', 'MDCK', 'tobacco', 'A. thaliana',
       'B', 'N2a', 'SCID', 'C57', 'larvae', 'Arabidopsis thaliana', 'BHK',
       'C57BL/6J mice', 'flies', 'HCT', 'L', 'day', 'C6/36', 'hamsters',
       'Ae. aegypti', 'sheep', 'CT26', 'sugarcane', 'piglets', '293',
       'rat', 'THP', 'rats', 'F', 'cotton', '0', 'WT', '5', '3', 'M',
       'COS', 'LNCaP', 'S2', 'D. melanogaster', 'MCF-7', 'BV',
       'zebrafish', 'Sf9'

In [70]:
word_list = ["N","S","A","6","7","T","nude","2","O","G","4","0","293","5","3","M","H","J","8","C.","19","B."]
sample_row = grouped[grouped['word'].isin(word_list)].groupby("word").apply(lambda df: df.sample(1))

In [71]:
with open("sentences.txt", "w") as file:
	for sentence in sample_row["text"].values:
		file.write(sentence + "\n")

---

In [4]:
d = pd.read_parquet(f"{path}/MEDIUM.parquet").explode("ner").dropna()
grouped = d.groupby("text").agg({"ner": lambda x: list(x)}).reset_index()
grouped["ner"] = grouped["ner"].apply(merge_entities)
grouped = grouped.explode("ner")
grouped = pd.concat(
                [grouped.drop(columns="ner"), grouped.ner.apply(pd.Series)], axis=1
            )
grouped["word"] = grouped.apply(
            lambda row: row["text"][row["start"] : row["end"]], axis=1
        )

In [7]:
grouped.value_counts("word").head(190).reset_index().word.values

array(['DMEM', 'PBS', 'LB', 'RPMI 1640', 'FBS', 'RPMI-1640', 'RPMI',
       'Dulbecco’s modified Eagle’s medium', 'fetal bovine serum', 'MEM',
       'TSB', 'serum', 'LB broth', 'bovine serum', 'DMEM/F12', 'PDA',
       'YPD', 'NGM', 'LB agar', "Dulbecco's modified Eagle's medium",
       'phosphate-buffered saline', 'Dulbecco’s Modified Eagle Medium',
       'M9', 'BHI', 'Luria-Bertani (', '10% FBS', 'serum-free', 'LB)',
       'Dulbecco’s modified Eagle’s medium (DMEM', 'RPMI1640',
       'Dulbecco’s Modified Eagle’s Medium', 'EMEM', 'RIPA', 'MRS',
       'Dulbecco’s modified Eagle medium', 'IMDM', 'agar', 'HBSS', 'MS',
       'complete', 'minimal', 'MRS broth', 'TSA', 'Tris-HCl', 'SD',
       'α-MEM', 'LB medium', 'phosphate buffered saline', 'complete DMEM',
       'glucose', 'F12', 'DMEM/F-12', 'Luria-Bertani', 'DMEM-F12',
       'Opti-MEM', 'MHB', 'potato dextrose agar',
       "Dulbecco's Modified Eagle Medium", 'LB) broth', 'DPBS',
       'Dulbecco’s Modified Eagle Medium (DMEM

In [8]:
word_list = ["lysogeny broth (","potato dextrose agar (","Luria-Bertani (","serum","10% FBS","LB)","Dulbecco’s modified Eagle’s medium (DMEM", "LB) broth","Roswell Park Memorial Institute (RPMI) 1640","HyClone","the","LB) agar","tryptic soy broth (TSB","Roswell Park Memorial Institute 1640","Roswell Park Memorial Institute","basal"]
sample_row = grouped[grouped['word'].isin(word_list)].groupby("word").apply(lambda df: df.sample(1))

In [9]:
with open("sentences.txt", "w") as file:
	for sentence in sample_row["text"].values:
		file.write(sentence + "\n")

---

In [10]:
d = pd.read_parquet(f"{path}/PHENOTYPE.parquet").explode("ner").dropna()
grouped = d.groupby("text").agg({"ner": lambda x: list(x)}).reset_index()
grouped["ner"] = grouped["ner"].apply(merge_entities)
grouped = grouped.explode("ner")
grouped = pd.concat(
                [grouped.drop(columns="ner"), grouped.ner.apply(pd.Series)], axis=1
            )
grouped["word"] = grouped.apply(
            lambda row: row["text"][row["start"] : row["end"]], axis=1
        )

In [14]:
grouped.value_counts("word").head(200).reset_index().word.values

array(['Gram-negative', 'Gram-positive', 'pathogenic', 'endophytic',
       'virulent', 'non-pathogenic', 'non', 'probiotic', 'hypervirulent',
       'avirulent', 'gram-negative', 'thermophilic', 'gram-positive',
       'multidrug-resistant', 'anaerobic', 'drought', 'motile',
       'mesophilic', 'aerobic', 'salt', 'nonpathogenic', 'non-motile',
       'halophilic', 'toxigenic', 'free-living', 'mucoid', 'multidrug',
       'rod-shaped', 'nitrogen-fixing', 'salt-tolerant', 'resistant',
       'highly pathogenic', 'MDR', 'drug', 'non-', 'auxotrophic',
       'nitrogen fixation', 'susceptible', 'commensal', 'tolerant',
       'cold', 'Gram negative', 'Gram positive', 'yellow', '-', 'white',
       'positive', 'green', 'invasive', 'uropathogenic', 'halotolerant',
       'PGPR', 'non-toxigenic', 'heat', 'cellulolytic', 'lytic',
       'sensitive', 'autotrophic', 'psychrophilic', 'thermostable',
       'heterotrophic', 'thermotolerant', 'fast-growing', 'prototrophic',
       'carbapenem', 'r

In [15]:
word_list = ["non","non-","N","-","methicillin","Gram","Gram-","multidrug-","self","triple-","cold-","R5"]
sample_row = grouped[grouped['word'].isin(word_list)].groupby("word").apply(lambda df: df.sample(1))

In [16]:
with open("sentences.txt", "w") as file:
	for sentence in sample_row["text"].values:
		file.write(sentence + "\n")

---

In [17]:
d = pd.read_parquet(f"{path}/MORPHOLOGY.parquet").explode("ner").dropna()
grouped = d.groupby("text").agg({"ner": lambda x: list(x)}).reset_index()
grouped["ner"] = grouped["ner"].apply(merge_entities)
grouped = grouped.explode("ner")
grouped = pd.concat(
                [grouped.drop(columns="ner"), grouped.ner.apply(pd.Series)], axis=1
            )
grouped["word"] = grouped.apply(
            lambda row: row["text"][row["start"] : row["end"]], axis=1
        )

In [22]:
grouped.value_counts("word").head(210).reset_index().word.values

array(['biofilm', 'biofilms', 'spores', 'spore', 'hyphae', 'sporulation',
       'hyphal', 'conidia', 'spheroids', 'colonies', 'rod', 'Biofilm',
       'planktonic', 'filamentous', 'colony', 'mycelial', 'Biofilms',
       'Spores', 'plaques', 'ascospores', 'aggregates', 'aerial hyphae',
       'sphere', 'Spore', 'filaments', 'flagella', 'mycelium',
       'aerial mycelium', 'spheres', 'zoospores', 'hypha', 'spindle',
       'swarming', 'rods', 'plaque', 'endospore', 'pellicle', 'mycelia',
       'microcolonies', 'multicellular', 'chlamydospores', 'sporulating',
       'coccoid', 'aerial mycelia', 'pseudopodia', 'sporulate',
       'spheroid', 'basidiospores', 'aerial', 'heterocysts', 'flagellar',
       'rosette', 'spore germination', 'flagellum', 'sporangiospores',
       'conidiophores', 'vegetative', 'cysts', 'capsule', 'filament',
       'sporulated', 'pilus', 'blastospores', 'fibrils', 'spiral',
       'pellicle biofilm', 'heterocyst', 'endospores', 'pellicles',
       'pseudohyph

In [23]:
word_list = ["vegetative","mushroom","non","mammosphere","forming rod", "amyloid", "quorum","fruiting"]
sample_row = grouped[grouped['word'].isin(word_list)].groupby("word").apply(lambda df: df.sample(1))

In [24]:
with open("sentences.txt", "w") as file:
	for sentence in sample_row["text"].values:
		file.write(sentence + "\n")

---

In [3]:
d = pd.read_parquet(f"{path}/EFFECT.parquet").explode("ner").dropna()
grouped = d.groupby("text").agg({"ner": lambda x: list(x)}).reset_index()
grouped["ner"] = grouped["ner"].apply(merge_entities)
grouped = grouped.explode("ner")
grouped = pd.concat(
                [grouped.drop(columns="ner"), grouped.ner.apply(pd.Series)], axis=1
            )
grouped["word"] = grouped.apply(
            lambda row: row["text"][row["start"] : row["end"]], axis=1
        )

In [4]:
grouped.value_counts("word").head(190).reset_index().word.values

array(['growth', 'apoptosis', 'formation', 'proliferation', 'cell death',
       'infection', 'stress', 'tumor growth', 'cell', 'cells',
       'cell proliferation', 'cytotoxic', 'death', 'production',
       'biofilm formation', 'survival', 'development', 'differentiation',
       'damage', 'viability', 'oxidative stress', 'cell growth', '-',
       'replication', 'activation', 'growth inhibition', 'inflammation',
       'migration', 'resistance', 'induction', 'metabolism',
       'cytotoxicity', 'accumulation', 'response', 'colony formation',
       'immune response', 'invasion', 'proliferative', 'cell viability',
       'plant growth', 'cell cycle arrest', 'activity', 'degradation',
       'cycle', 'gene expression', 'synthesis', 'expression', 'autophagy',
       'disease', 'antimicrobial', 'inflammatory', 'cell cycle', 'tumor',
       'metastasis', 'inflammatory response', 'uptake', 'release',
       'cell apoptosis', 'viral replication', 'ferroptosis', 'tolerance',
       'antipro

In [5]:
word_list = ["cell","cells","effect","-","cycle","activity","cell cycle","host","gene","acid","effects","genes","infections","cytokines","challenge","anti","flux","drug","membrane","biomass","system"]
sample_row = grouped[grouped['word'].isin(word_list)].groupby("word").apply(lambda df: df.sample(1))

In [6]:
with open("sentences.txt", "w") as file:
	for sentence in sample_row["text"].values:
		file.write(sentence + "\n")

---

In [3]:
d = pd.read_parquet(f"{path}/PROTEIN.parquet").explode("ner").dropna()
grouped = d.groupby("text").agg({"ner": lambda x: list(x)}).reset_index()
grouped["ner"] = grouped["ner"].apply(merge_entities)
grouped = grouped.explode("ner")
grouped = pd.concat(
                [grouped.drop(columns="ner"), grouped.ner.apply(pd.Series)], axis=1
            )
grouped["word"] = grouped.apply(
            lambda row: row["text"][row["start"] : row["end"]], axis=1
        )

In [4]:
grouped.value_counts("word").head(310).reset_index().word.values

array(['GFP', 'anti', 'IL-6', 'ACE2', 'IL', 'TNF-α', 'p53', 'NF-κB',
       'IL-1β', 'S', 'EGFR', 'spike', 'β-catenin', 'Akt', 'AKT',
       'E-cadherin', 'IL-8', 'HA', 'IL-10', 'GST', 'His', 'PI3K', 'STAT3',
       'IFN-γ', 'RBD', 'PD-L1', 'IFN', 'ERK', 'caspase-3', 'Bcl-2', 'TNF',
       'N', 'mTOR', 'Bax', 'VEGF', 'HER2', 'AR', 'HIF-1α', 'TLR4', 'SOD',
       'CD4', 'p21', 'JNK', 'CD44', 'ERK1/2', 'protease', 'vimentin',
       'NF', 'TGF', 'β', 'P-gp', 'mCherry', 'Nrf2', 'IgG', 'CXCR4',
       'TMPRSS2', 'PTEN', 'MMP-9', 'α', 'Wnt', 'FLAG', 'CAT',
       'β-galactosidase', 'S1', 'TNFα', 'PD-1', 'p38', 'PARP', 'AMPK',
       'N-cadherin', 'M^pro', 'CD3', 'S protein', 'c', 'COX-2', 'YAP',
       '2', 'E', 'EGFP', 'iNOS', 'Cas9', 'LC3', 'NS1', 'Env', 'TLR2',
       'HLA', 'ERα', 'p', 'catalase', 'NP', 'IL-12', 'Fc', 'MAPK',
       'gp120', 'MMP-2', 'PE', 'CAR', 'RdRp', 'ZO-1', 'Flag', 'STAT1',
       'p65', 'p62', 'IL-2', 'MCP-1', 'IL-', 'IFNγ', '1', 'IL-4', 'YFP',
       'cyclin D1',

In [5]:
word_list = ["M","C","6","N","1","H","β-","3","5","pro","I","P","p","2","E","B","-"]
sample_row = grouped[grouped['word'].isin(word_list)].groupby("word").apply(lambda df: df.sample(1))

In [6]:
with open("sentences.txt", "w") as file:
	for sentence in sample_row["text"].values:
		file.write(sentence + "\n")

---

In [7]:
d = pd.read_parquet(f"{path}/GENUS.parquet").explode("ner").dropna()
grouped = d.groupby("text").agg({"ner": lambda x: list(x)}).reset_index()
grouped["ner"] = grouped["ner"].apply(merge_entities)
grouped = grouped.explode("ner")
grouped = pd.concat(
                [grouped.drop(columns="ner"), grouped.ner.apply(pd.Series)], axis=1
            )
grouped["word"] = grouped.apply(
            lambda row: row["text"][row["start"] : row["end"]], axis=1
        )

In [15]:
grouped.value_counts("word").head(180).reset_index().word.values

array(['Lactobacillus', 'Wolbachia', 'Bacteroides', 'Ruminococcaceae',
       'Streptomyces', 'Pseudomonas', 'Bacillus', 'Ruminococcus',
       'Prevotella', 'Bifidobacterium', 'Salmonella', 'Clostridium',
       'Streptococcus', 'Prevotellaceae', 'Faecalibacterium',
       'Akkermansia', 'Shigella', 'Synechococcus', 'Acinetobacter',
       'Escherichia', 'Enterococcus', 'Ruminiclostridium', 'Vibrio',
       'Roseburia', 'Blautia', 'Sphingomonas', 'Burkholderia',
       'Desulfovibrio', 'Staphylococcus', 'Campylobacter', 'Rhodococcus',
       'Lachnoclostridium', 'Parabacteroides', 'Prochlorococcus',
       'Treponema', 'Lactococcus', 'Klebsiella', 'Clostridia',
       'Coprococcus', 'Enterobacter', 'Alloprevotella', 'Turicibacter',
       'Agrobacterium', 'Helicobacter', 'Rickettsia',
       'Phascolarctobacterium', 'Eubacterium', 'Flavobacterium',
       'Oscillospira', 'Odoribacter', 'Rhizobium', 'Romboutsia',
       'Bacteroidetes', 'Alistipes', 'Corynebacterium', 'Veillonella',
  

In [ ]:
word_list = [""]
sample_row = grouped[grouped['word'].isin(word_list)].groupby("word").apply(lambda df: df.sample(1))

In [ ]:
with open("sentences.txt", "w") as file:
	for sentence in sample_row["text"].values:
		file.write(sentence + "\n")

---

In [16]:
d = pd.read_parquet(f"{path}/DISEASE.parquet").explode("ner").dropna()
grouped = d.groupby("text").agg({"ner": lambda x: list(x)}).reset_index()
grouped["ner"] = grouped["ner"].apply(merge_entities)
grouped = grouped.explode("ner")
grouped = pd.concat(
                [grouped.drop(columns="ner"), grouped.ner.apply(pd.Series)], axis=1
            )
grouped["word"] = grouped.apply(
            lambda row: row["text"][row["start"] : row["end"]], axis=1
        )

In [23]:
grouped.value_counts("word").head(200).reset_index().word.values

array(['tumor', 'breast cancer', 'cancer', 'COVID', 'melanoma',
       'pandemic', 'CRC', 'HCC', 'colon cancer', 'prostate cancer',
       'tumors', 'glioma', 'ovarian cancer', 'lung cancer', 'NSCLC',
       'glioblastoma', 'SARS', 'TNBC', 'breast', 'GBM', 'neuroblastoma',
       'pneumonia', 'pancreatic cancer', 'influenza', 'tumour',
       'gastric cancer', 'colorectal cancer', 'disease', 'GC', 'ARDS',
       'leukemia', 'AML', 'osteosarcoma', 'diarrhea', 'PCa', 'BC',
       'hepatocellular carcinoma', 'metastasis', 'outbreak', 'colitis',
       'PDAC', 'lung adenocarcinoma', 'colon', 'lung', 'sepsis',
       'cervical cancer', 'AD', 'colon carcinoma', 'adenocarcinoma',
       'hepatoma', 'metastatic', 'bladder cancer',
       'breast adenocarcinoma', 'obesity', 'human breast cancer',
       'carcinoma', 'PD', 'colorectal adenocarcinoma', 'prostate',
       'colon adenocarcinoma', 'OSCC', 'MM', 'lymphoma', '19 pandemic',
       'NPC', 'acute respiratory syndrome', 'infection', 'TB',

In [24]:
word_list = ["PCa","COVID-","19","19 disease","19 pneumonia"]
sample_row = grouped[grouped['word'].isin(word_list)].groupby("word").apply(lambda df: df.sample(1))

In [25]:
with open("sentences.txt", "w") as file:
	for sentence in sample_row["text"].values:
		file.write(sentence + "\n")

## Check strains

In [17]:
d = pd.read_parquet(f"{path}/STRAIN/strains.parquet")

In [81]:
d[d.word.str.contains("escherichia coli ")].value_counts("word")

Series([], Name: count, dtype: int64)

In [79]:
words = ["=lmg 31545t", "=nbrc 113514t", "=nbrc 109120t), ilumatobacter fluminis ym22-133t","=stm 815"]

In [75]:
sample_row = d[d['word'].isin(words)].groupby("word").apply(lambda df: df.sample(1))

In [76]:
with open("sentences.txt", "w") as file:
	for sentence in sample_row["text"].values:
		file.write(sentence + "\n")